In [1]:
import leafmap
import geopandas as gpd
import numpy as np
import pathlib

from geospatial_tools import DATA_DIR
from shapely.geometry import Polygon

## Base data

The USA polygon is base off 2018's `cb_2018_us_nation_5m` shapefile, taken from here: 
https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

It was then processed using QGIS to keep only the contiguous states, without any islands.

The Sentinel 2 grid was taken from the kml file found here: 
https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/data-products

It was then processed using QGIS to keep only the grid cells that overlap with the 
contiguous states, meaning the polygon layer which is described just above.

Since our area of study is quite large, the `EPSG:5070` projection was chosen, as it
covers the whole area, introduces minimal distortion while preserving area.

In [2]:
USA_POLYGON_FILE = DATA_DIR / "usa/usa_polygon_5070.gpkg"
S2_USA_GRID_FILE = DATA_DIR / "usa/s2_grid_usa_polygon_5070.gpkg"

In [3]:
usa_polygon = gpd.read_file(USA_POLYGON_FILE)
s2_grid = gpd.read_file(S2_USA_GRID_FILE)

In [4]:
usa_polygon

,AFFGEOID,GEOID,NAME,geometry
0,0100000US,US,United States,"MULTIPOLYGON (((-2123555.702 3120381.564, -212..."


In [5]:
s2_grid

,name,folders,description,altitude,alt_mode,time_begin,time_end,time_when,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1386334.944 2487548.770 0.0...
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-976300.478 2523767.452 0.00...
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-960099.705 2622374.255 0.00...
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1008622.024 2325748.358 0.0...
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-992478.385 2424861.340 0.00...
...,...,...,...,...,...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1515431.586 2304192.826 0.0...
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1448525.813 2089886.667 0.0...
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1371006.917 2586590.133 0.0...
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1355793.563 2685354.080 0.0...


In [6]:
# m = leafmap.Map(center=[40, -98], zoom=4)
# 
# # In blue, the USA polygon
# m.add_gdf(usa_polygon, layer='usa')
# # In red, the Sentinel 2 grid
# m.add_gdf(s2_grid, layer='s2_grid', style={"color": "red"})
# 
# m

## Creating our inference grid

From this, we want to create a grid of square polygons with which we will later on
query the [Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
Sentinel 2 dataset and clip the selected Sentinel 2 images.

In [7]:
import time
import os

from geospatial_tools.vector import create_vector_grid, create_vector_grid_parallel


In [8]:
grid_size = 5000
bbox = usa_polygon.total_bounds

## Creating our inference grid

From this, we want to create a grid of square polygons with which we will later on
query the [Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
Sentinel 2 dataset and clip the selected Sentinel 2 images.

In [11]:
start = time.time()
print("Starting processing for [create_vector_grid]")
grid = create_vector_grid(bounding_box=bbox, grid_size=grid_size, crs="EPSG:5070")
stop = time.time()
print(f"Number of polygons: {len(grid)}")
print(f"Time taken to create grid: {stop - start}")

In [12]:
start = time.time()
print("Starting write for [create_vector_grid_parallel]")
grid.to_file(filename="test_write_vector_grid", driver="GPKG")
stop = time.time()
print(f"Time taken to write parallel grid to file: {stop - start}")

In [13]:
start = time.time()
print("Starting processing for [create_vector_grid_parallel]")
grid_parallel = create_vector_grid_parallel(bounding_box=bbox, grid_size=grid_size, crs="EPSG:5070")
stop = time.time()
print(f"Number of polygons: {len(grid)}")
print(f"Time taken to create parallel grid: {stop - start}")

Time taken to create grid: 119.25849294662476


,geometry
0,"POLYGON ((-2356113.743 310919.600, -2355113.74..."
1,"POLYGON ((-2355113.743 310919.600, -2354113.74..."
2,"POLYGON ((-2354113.743 310919.600, -2353113.74..."
3,"POLYGON ((-2353113.743 310919.600, -2352113.74..."
4,"POLYGON ((-2352113.743 310919.600, -2351113.74..."
...,...
13175820,"POLYGON ((2253886.257 3164919.600, 2254886.257..."
13175821,"POLYGON ((2254886.257 3164919.600, 2255886.257..."
13175822,"POLYGON ((2255886.257 3164919.600, 2256886.257..."
13175823,"POLYGON ((2256886.257 3164919.600, 2257886.257..."


In [15]:
start = time.time()
print("Starting write for [create_vector_grid_parallel]")
grid_parallel.to_file(filename="test_write_grid_parallel", driver="GPKG")
stop = time.time()
print(f"Time taken to write parallel grid to file: {stop - start}")

In [19]:
from concurrent.futures import ProcessPoolExecutor
import fiona
import shutil

In [24]:
def write_chunk(chunk, filename, mode='w'):
    """
    Write a single chunk to the GeoPackage.

    Parameters:
    chunk (GeoDataFrame): The chunk to write.
    filename (str): The filename to write to.
    mode (str): The mode to open the file with ('w' for write, 'a' for append).

    Returns:
    None
    """
    with fiona.open(filename, mode=mode, driver='GPKG', schema=chunk.schema, crs=chunk.crs) as layer:
        layer.writerecords(chunk.iterfeatures())

def to_file_parallel(gdf, filename, chunk_size=10000):
    """
    Save GeoDataFrame to a file in parallel.

    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame to save.
    filename (str): The filename to save to.
    chunk_size (int): The number of rows per chunk.
    num_workers (int): The number of parallel workers.

    Returns:
    None
    """
    if os.path.exists(filename):
        os.remove(filename)

    chunks = [gdf.iloc[i:i + chunk_size] for i in range(0, len(gdf), chunk_size)]
    
    with ProcessPoolExecutor() as executor:
        # First chunk in write mode
        executor.submit(write_chunk, chunks[0], filename, 'w')
        
        # Remaining chunks in append mode
        for chunk in chunks[1:]:
            executor.submit(write_chunk, chunk, filename, 'a')

In [23]:
to_file_parallel(grid_parallel, "grid_parallel3.gpkg")